In [0]:
%fs ls "mnt/dataengproj/silver/SalesLT/"

path,name,size,modificationTime
dbfs:/mnt/dataengproj/silver/SalesLT/Address/,Address/,0,1731075237000
dbfs:/mnt/dataengproj/silver/SalesLT/Customer/,Customer/,0,1731075250000
dbfs:/mnt/dataengproj/silver/SalesLT/CustomerAddress/,CustomerAddress/,0,1731075253000
dbfs:/mnt/dataengproj/silver/SalesLT/Product/,Product/,0,1731075256000
dbfs:/mnt/dataengproj/silver/SalesLT/ProductCategory/,ProductCategory/,0,1731075259000
dbfs:/mnt/dataengproj/silver/SalesLT/ProductDescription/,ProductDescription/,0,1731075261000
dbfs:/mnt/dataengproj/silver/SalesLT/ProductModel/,ProductModel/,0,1731075264000
dbfs:/mnt/dataengproj/silver/SalesLT/ProductModelProductDescription/,ProductModelProductDescription/,0,1731075266000
dbfs:/mnt/dataengproj/silver/SalesLT/SalesOrderDetail/,SalesOrderDetail/,0,1731075268000
dbfs:/mnt/dataengproj/silver/SalesLT/SalesOrderHeader/,SalesOrderHeader/,0,1731075271000


###Changing column names in all tables from pascal case to different format

In [0]:
#collecting table names
table_names = []

for i in dbutils.fs.ls("dbfs:/mnt/dataengproj/silver/SalesLT/"):
  table_names.append(i.name.split('/')[0])

In [0]:
table_names

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
from pyspark.sql.functions import col, regexp_replace

for tbl_name in table_names:
  path = "dbfs:/mnt/dataengproj/silver/SalesLT/" + tbl_name
  print("modifying table: " + tbl_name)
  df = spark.read.format("delta").load(path)

  #Get column names
  col = df.columns

  for oldcol_name in col:
    #set new col name -> ColumnName to Column_Name
    newcol_name = "".join(["_" + char if char.isupper() and not oldcol_name[i-1].isupper() else char for i,char in enumerate(oldcol_name)]).lstrip("_") #to remove the first underscore from column name

    #renaming columns
    df = df.withColumnRenamed(oldcol_name,newcol_name)
 
  #save renamed dataframes in delta format to gold container
  output_path = "dbfs:/mnt/dataengproj/gold/SalesLT/"+tbl_name+"/"
  df.write.format("delta").mode("overwrite").option("mergeSchema","true").save(output_path)


modifying table: Address
modifying table: Customer
modifying table: CustomerAddress
modifying table: Product
modifying table: ProductCategory
modifying table: ProductDescription
modifying table: ProductModel
modifying table: ProductModelProductDescription
modifying table: SalesOrderDetail
modifying table: SalesOrderHeader


####Column names has been changed for all tables and moved to gold layer

In [0]:
display(df)